### 1. read JSON file(test data) and select the reviews from one hotel and convert it to a dataframe


In [95]:
import pandas as pd

# Read JSON file into a DataFrame
with open('scraped_data_5.json', 'r') as f:
    df = pd.read_json(f)

# get all the business names to a list
business_names = df['business_name'].tolist()
"""
business_names =
['Holiday Inn Express Singapore Katong, an IHG Hotel',
 'Ibis Budget Singapore Pearl',
 'ibis Styles Singapore Albert',
 'Village Hotel Albert Court by Far East Hospitality',
 'lyf Farrer Park Singapore']
 
 """
# a function to get all reviews from one hotel
def get_hotel_reviews(df, hotel_name):
    # Get the row of the DataFrame where the 'business_name' column is equal to the hotel_name
    hotel_df = df[df['business_name'] == hotel_name]

    # Get the review list from that hotel: reviews column first row
    reviews = hotel_df['reviews'].iloc[0]

    # Merge all rows of the 'reviews' column into one list
    reviews_list = [review for review in reviews]

    # Create a DataFrame from the list of dictionaries
    review_df = pd.DataFrame(reviews_list)

    #only keep 3 columns we needed, and rename the columns 'Negative_Review', 'Positive_Review', 'Review_Date',
    reviews_df_clean = review_df[['review_date', 'review_liked', 'review_disliked']]
    reviews_df_clean = reviews_df_clean.rename(columns={'review_date': 'Review_Date', 'review_liked': 'Positive_Review', 'review_disliked': 'Negative_Review'})

    reviews_df_clean['Hotel_Name'] = 'lyf Farrer Park Singapore'

    print(f"Created DataFrame with {len(reviews_df_clean)} rows.")
    # Return the DataFrame of reviews
    return reviews_df_clean

# Get the reviews for the 'lyf Farrer Park Singapore' hotel
review_df = get_hotel_reviews(df, 'lyf Farrer Park Singapore')
review_df.head()

Created DataFrame with 1623 rows.


,Review_Date,Positive_Review,Negative_Review,Hotel_Name
0,"May 26, 2023","The ease of the stay, from check in to check o...",Having to leave 😭,lyf Farrer Park Singapore
1,"June 24, 2023","Great location, supermarket in front and a lot...","The neighbors were to noisy at midnight, screa...",lyf Farrer Park Singapore
2,"June 22, 2023",Location and gym.,"Noisy at night. Some students stayed over, but...",lyf Farrer Park Singapore
3,"June 22, 2023","Room is big, great view.",The bathroom layout gives no privacy at all. N...,lyf Farrer Park Singapore
4,"June 22, 2023",Location is excellent. Well withing the reach ...,I think the frequency and quality of cleaning ...,lyf Farrer Park Singapore


### 2. clean the review text for testing data

In [96]:
import numpy as np
from langdetect import detect
# Positive_Review: invalid reviews are with the following strings:'There are no comments available for this review', 'everything'
test_pos_invalid_content = 'there are no comments available for this review|everything'
# Negative_Review: invalid reviews are with the following strings:
test_neg_invalid_content = 'nothing|n/a|none'


# define a function to check if a string is in English
def is_english(text):
    if not text: return True # an empty string is considered English
    try:
        lang = detect(text)
        return lang == 'en'
    except:
        return False

# define a function to do basic clean of the review text
def clean_text(review_df, column_name, invalid_content_str):
    # create the docstring for this function
    """
    # Input:
    # the review_df with 4 columns: 'Negative_Review', 'Positive_Review', 'Review_Date', 'Hotel_Name'
    # column_name: 'Negative_Review' or 'Positive_Review'
    # 
    # Output: 
    # a DataFrame with the same columns as review_df, but with invalid reviews removed
    # """

    # filter out non-English reviews from the column
    english_reviews = review_df[review_df[column_name].apply(is_english)]
    english_reviews

    # convert to lowercase
    english_reviews[column_name] = english_reviews[column_name].str.lower()

    # apply strip() to remove leading and trailing whitespaces
    english_reviews[column_name] = english_reviews[column_name].apply(lambda x: x.strip())

    # if the text contain invalid_content_str, replace the text with NAN
    english_reviews.loc[english_reviews[column_name].str.contains(invalid_content_str, case=False), column_name] = np.nan
    
    # if the text is empty, replace the text with NAN
    english_reviews.loc[english_reviews[column_name] == '', column_name] = np.nan  
    return english_reviews

# define a function to clean the review text from one hotel dateframe(testing data)
def clean_hotel_reviews(review_df, test_pos_invalid_content, test_neg_invalid_content):
    # clean the positive reviews
    cleaned_pos_df = clean_text(review_df, 'Positive_Review', test_pos_invalid_content)
    # clean the negative reviews
    cleaned_neg_df = clean_text(cleaned_pos_df, 'Negative_Review', test_neg_invalid_content)
    return cleaned_neg_df

    



In [97]:
cleaned_review_df = clean_hotel_reviews(review_df, test_pos_invalid_content, test_neg_invalid_content)

/var/folders/w2/5y09ynfn1dn9nzmzn5n1b4740000gp/T/ipykernel_82851/3307416861.py:35: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  english_reviews[column_name] = english_reviews[column_name].str.lower()
/var/folders/w2/5y09ynfn1dn9nzmzn5n1b4740000gp/T/ipykernel_82851/3307416861.py:38: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  english_reviews[column_name] = english_reviews[column_name].apply(lambda x: x.strip())


In [101]:
cleaned_review_df

,Review_Date,Positive_Review,Negative_Review,Hotel_Name
0,"May 26, 2023","the ease of the stay, from check in to check o...",having to leave 😭,lyf Farrer Park Singapore
1,"June 24, 2023","great location, supermarket in front and a lot...","the neighbors were to noisy at midnight, screa...",lyf Farrer Park Singapore
2,"June 22, 2023",location and gym.,"noisy at night. some students stayed over, but...",lyf Farrer Park Singapore
4,"June 22, 2023",location is excellent. well withing the reach ...,i think the frequency and quality of cleaning ...,lyf Farrer Park Singapore
6,"June 21, 2023",location was perfect near to farrer park stati...,bathroom and wc was inside the room there was ...,lyf Farrer Park Singapore
...,...,...,...,...
1618,"March 22, 2023",NaN,NaN,lyf Farrer Park Singapore
1619,"March 6, 2023",NaN,NaN,lyf Farrer Park Singapore
1620,"January 26, 2023",NaN,NaN,lyf Farrer Park Singapore
1621,"December 20, 2022",NaN,NaN,lyf Farrer Park Singapore


In [99]:
cleaned_review_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1324 entries, 0 to 1622
Data columns (total 4 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   Review_Date      1324 non-null   object
 1   Positive_Review  473 non-null    object
 2   Negative_Review  328 non-null    object
 3   Hotel_Name       1324 non-null   object
dtypes: object(4)
memory usage: 51.7+ KB
